# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
#%matplotlib notebook

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

#Configure gmaps
gmaps.configure(api_key=g_key)
#print(g_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
city_wx_df = pd.read_csv("../output_data/cities_weather.csv")
city_wx_df

,City,Country,City ID,lon,lat,temperature,max temperature,humidity,clouds,windspeed
0,Ushuaia,AR,3833367,-68.3000,-54.8000,46.40,46.40,81,75,36.82
1,Tasiilaq,GL,3424607,-37.6368,65.6145,23.00,23.00,74,20,2.48
2,Padang,ID,1633419,100.3543,-0.9492,86.00,86.00,66,20,4.61
3,Bluff,NZ,2206939,168.3333,-46.6000,57.00,57.00,97,100,8.99
4,Mataura,NZ,6201424,168.8643,-46.1927,57.00,57.00,97,100,8.99
...,...,...,...,...,...,...,...,...,...,...
542,Kirensk,RU,2022083,108.1119,57.7853,1.78,1.78,89,100,6.73
543,Bandarbeyla,SO,64814,50.8122,9.4942,76.30,76.30,68,24,13.53
544,Irbeyskoye,RU,1505529,95.4522,55.6428,-1.30,-1.30,90,83,4.00
545,Dzaoudzi,YT,921900,45.2699,-12.7887,80.60,80.60,83,40,6.91


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
city_list = city_wx_df['City'].tolist()
humidity_rate = city_wx_df['humidity'].astype(float)

marker_locations = city_wx_df[['lat', 'lon']]

# Thank you to TA Chris Howard for helping with the zoom level and center for gmaps!
us_coordinates = (0.0, 0.0)
fig = gmaps.figure(center=us_coordinates, zoom_level=2)

markers = gmaps.marker_layer(marker_locations,
                            info_box_content=[f'Ideal city: {city}' for city in city_list])
heat_layer = gmaps.heatmap_layer(marker_locations, weights=humidity_rate,
                               dissipating=False, max_intensity=100,
                               point_radius=1)
fig.add_layer(markers)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the DataFrame to find ideal weather condition
# Max temperature lower than 80 degrees but higher than 70
ideal_df = city_wx_df[city_wx_df['max temperature'] < 80]
ideal_df = ideal_df[ideal_df['max temperature'] > 70]

# Wind speed less than 10 mph
ideal_df = ideal_df[ideal_df['windspeed'] < 10]

# Zero cloudiness
ideal_df = ideal_df[ideal_df['clouds'] == 0]

ideal_df.dropna()

ideal_df

,City,Country,City ID,lon,lat,temperature,max temperature,humidity,clouds,windspeed
32,Arraial do Cabo,BR,3471451,-42.0278,-22.9661,74.41,74.41,88,0,9.40
43,Sur,OM,286245,59.5289,22.5667,79.21,79.21,43,0,3.20
68,São Filipe,CV,3374210,-24.4956,14.8961,71.71,71.71,69,0,7.27
82,Petrolina,BR,3392242,-40.5008,-9.3986,75.20,75.20,94,0,1.14
193,Kontagora,NG,2334008,5.4695,10.3999,71.46,71.46,20,0,7.36
253,Lázaro Cárdenas,MX,4026082,-102.2000,17.9583,71.64,71.64,88,0,4.16
326,Santa Isabel,BR,3450188,-46.2214,-23.3156,72.48,75.00,78,0,4.61
434,Narendranagar,IN,1261806,78.3000,30.1667,73.36,73.36,22,0,2.17
463,Ziyang,CN,1790863,117.8547,29.2497,70.56,70.56,50,0,5.64
500,Nizwá,OM,286987,57.5333,22.9333,78.12,78.12,24,0,1.36


In [6]:
ideal_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12 entries, 32 to 536
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   City             12 non-null     object 
 1   Country          12 non-null     object 
 2   City ID          12 non-null     int64  
 3   lon              12 non-null     float64
 4   lat              12 non-null     float64
 5   temperature      12 non-null     float64
 6   max temperature  12 non-null     float64
 7   humidity         12 non-null     int64  
 8   clouds           12 non-null     int64  
 9   windspeed        12 non-null     float64
dtypes: float64(5), int64(3), object(2)
memory usage: 1.0+ KB


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = ideal_df[['City','Country','lat','lon']]
hotel_df['Hotel Name'] = ""
hotel_df['Rating'] = ""
hotel_df.reset_index(level=None,drop=True,inplace=True)
hotel_df
 

C:\Users\Sharon\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Sharon\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,City,Country,lat,lon,Hotel Name,Rating
0,Arraial do Cabo,BR,-22.9661,-42.0278,,
1,Sur,OM,22.5667,59.5289,,
2,São Filipe,CV,14.8961,-24.4956,,
3,Petrolina,BR,-9.3986,-40.5008,,
4,Kontagora,NG,10.3999,5.4695,,
5,Lázaro Cárdenas,MX,17.9583,-102.2000,,
6,Santa Isabel,BR,-23.3156,-46.2214,,
7,Narendranagar,IN,30.1667,78.3000,,
8,Ziyang,CN,29.2497,117.8547,,
9,Nizwá,OM,22.9333,57.5333,,


In [8]:
#
#coords = "-33.9180,25.5701"
target_type = 'lodging'
radius = 5000
# Build URL using the Google Maps API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#city_list = ['Port Elizabeth']   #, 'New Norfolk']
#coords = '-33.9180,25.5701'    #,-42.7826	147.0587]

nearest_hotels = []
location_list = []
hotel_ratings = []
heat_map_city_list = []
heat_map_locations = []

# Thanks to tutor Ruchi Chandra for suggesting I use df.iterrows() here!!
for index, row in  hotel_df.iterrows():
    print(row['City'])
    print('*********************************************************')
    #row = hotel_df.loc[hotel_df['City'] == city]
    ##print(row)
    
    latitude = row['lat']
    longitude = row['lon']
    
    coords = f"{latitude},{longitude}"
    location_list.append(coords)
    
    #print (coords)
    params = {
        'location':coords,
        'types': target_type,
        'radius': radius,
        'key': g_key
    }
    
    hotels = requests.get(base_url, params).json()
    
    #print(json.dumps(hotels, indent=4, sort_keys=True))
    
    try:
        hotel_name = hotels['results'][0]['name']
        #hotel_df.loc[index,'Hotel Name'] = hotels['results'][0]['name']
        hotel_df.loc[index,'Hotel Name'] = hotel_name
        
        rating = hotels['results'][0]['rating']
        #hotel_df.loc[index,'Rating'] = hotels['results'][0]['rating']
        hotel_df.loc[index,'Rating'] = rating
    except:
        fred = row['City']
        print(f'EXCEPTION:{fred} is missing ?')
 
    
hotel_df

Arraial do Cabo
*********************************************************


C:\Users\Sharon\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Sur
*********************************************************
São Filipe
*********************************************************
Petrolina
*********************************************************
Kontagora
*********************************************************
EXCEPTION:Kontagora is missing ?
Lázaro Cárdenas
*********************************************************
Santa Isabel
*********************************************************
Narendranagar
*********************************************************
Ziyang
*********************************************************
EXCEPTION:Ziyang is missing ?
Nizwá
*********************************************************
Taquaritinga
*********************************************************
São João da Barra
*********************************************************


,City,Country,lat,lon,Hotel Name,Rating
0,Arraial do Cabo,BR,-22.9661,-42.0278,Pousada Porto Praia,4.3
1,Sur,OM,22.5667,59.5289,Sur Plaza Hotel,3.8
2,São Filipe,CV,14.8961,-24.4956,Tortuga B&B,4.4
3,Petrolina,BR,-9.3986,-40.5008,JB Hotel,4
4,Kontagora,NG,10.3999,5.4695,Queens Guest Inn,
5,Lázaro Cárdenas,MX,17.9583,-102.2000,Collection O Hotel Quinta Antigua,4.3
6,Santa Isabel,BR,-23.3156,-46.2214,Chácara Sombra do Altíssimo,4.9
7,Narendranagar,IN,30.1667,78.3000,Ananda In The Himalayas,4.4
8,Ziyang,CN,29.2497,117.8547,Wuyuan Great Wall Hotel,
9,Nizwá,OM,22.9333,57.5333,Orient Apartments,4.5


In [9]:
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        12 non-null     object 
 1   Country     12 non-null     object 
 2   lat         12 non-null     float64
 3   lon         12 non-null     float64
 4   Hotel Name  12 non-null     object 
 5   Rating      12 non-null     object 
dtypes: float64(2), object(4)
memory usage: 704.0+ bytes


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# I can't get this line of code to work, so I am going back to what worked for me earlier
locations = hotel_df[["lat", "lon"]]




In [11]:
# Add marker layer ontop of heat map

# Thank you to TA Chris Howard for helping with the info_box_content=hotel_info on the marker layer!
fig = gmaps.figure()
hotel_markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
heat_layer = gmaps.heatmap_layer(locations, weights=None,
                                dissipating=False, max_intensity=100,
                                point_radius=1)
fig.add_layer(hotel_markers)
fig.add_layer(heat_layer)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))